In [1]:
include("QCBM.jl")

mmd_gradient

In [2]:
nlayers = 10
nbit = 9
maxiter = 10
learning_rate = 0.1

kernel = RBFKernel(nbit, [2.0], false)
params = reverse!(vec(readdlm("theta-cl-bs.dat")))
target = readdlm("wave_complex.dat")[:, 1] + im * readdlm("wave_complex.dat")[:, 2]
ptrain = randn(1<<nbit);

In [3]:
cnot_pair = [
    (1, 7),
    (2, 8),
    (4, 7),
    (5, 3),
    (6, 0),
    (6, 3),
    (6, 7),
    (8, 5),
]

cnot_pair = map(x->(x[1]+1, x[2]+1), cnot_pair)

8-element Array{Tuple{Int64,Int64},1}:
 (2, 8)
 (3, 9)
 (5, 8)
 (6, 4)
 (7, 1)
 (7, 4)
 (7, 8)
 (9, 6)

In [7]:
layer1 = chain(rot(:X), rot(:Z)) |> cache(2, recursive=true) |> roll
layers = MatrixBlock[]
push!(layers, layer1(nbit))

layer = chain(rot(:Z), rot(:X), rot(:Z)) |> cache(2, recursive=true) |> roll

for i = 1:nlayers
    push!(layers, cache(entangler(nbit, cnot_pair)))
    push!(layers, layer(nbit))
end

circuit = chain(layers...)

import Base: empty!
function empty!(layers::Vector{MatrixBlock})
    for each in layers
        empty!(each)
    end
    seq
end

empty! (generic function with 35 methods)

In [8]:
reg = zero_state(9)

Default Register (CPU, Complex{Float64}):
    total: 9
    batch: 1
    active: 9

In [9]:
zero_state(9) |> dispatch!(layer1(9), params) |> state

512×1 Array{Complex{Float64},2}:
  -1.09586e-7+3.26171e-8im 
 -9.11806e-10-2.89597e-9im 
  -8.81625e-9-1.0451e-7im  
   2.77114e-9-2.77752e-10im
 -3.96591e-11-1.42365e-11im
  3.61417e-13-1.05893e-12im
  2.01955e-11-3.29564e-11im
  8.83458e-13+5.22435e-13im
     0.015876-0.0108946im  
  0.000295895+0.000416969im
   0.00681764+0.0162933im  
 -0.000429752+0.000187823im
    7.0922e-6+2.3064e-7im  
             ⋮             
  1.36356e-10-3.11992e-10im
  8.34062e-12+3.48998e-12im
   2.5471e-10+1.80751e-10im
 -4.69265e-12+6.83831e-12im
    0.0791955+0.133923im   
  -0.00352267+0.00215867im 
    -0.135069+0.0460993im  
  -0.00128042-0.00356692im 
  -5.71843e-6+5.70529e-5im 
  -1.51719e-6-1.27987e-7im 
  -5.01686e-5-1.57957e-5im 
   3.98425e-7-1.33862e-6im 

In [17]:
state(reg) - target

512×1 Array{Complex{Float64},2}:
    0.0324683+0.280684im  
   -0.0573191+0.00380631im
  -0.00792277+0.022652im  
    0.0314004+0.0398405im 
   -0.0289954-0.0452687im 
   -0.0158421+0.059139im  
    0.0330286+0.0120057im 
    -0.299654-0.00125209im
    0.0291764+0.0182597im 
    0.0232386-0.0180867im 
  -0.00539307-0.0353238im 
    0.0150651+0.00205575im
   -0.0264854-0.0368902im 
             ⋮            
    0.0181784-0.00287241im
    0.0532593+0.0370849im 
     0.031076+0.0447798im 
  -0.00126424+0.0286751im 
     0.316653+0.100374im  
   -0.0381707+0.0112755im 
   -0.0205379-0.0133153im 
 -0.000536247+0.0230263im 
   -0.0143036+0.0595423im 
    0.0267782-0.0240496im 
   -0.0165767-0.00635973im
  -0.00928917+0.292749im  

In [9]:
l = layer

279-element Array{Float64,1}:
  6.23009   
 -0.0157379 
  4.79861   
  0.20513   
  6.28245   
  5.34632   
  3.1416    
  1.25866   
  4.71893   
  1.04387   
  2.83396   
  4.53111   
  5.31671   
  ⋮         
  3.44257   
  3.12023   
  1.81704   
  1.7664    
  1.42521   
  4.71157   
  5.62927   
 -4.3937e-5 
  3.14151   
  4.45657   
  2.20976   
  0.00101785

In [10]:
l = roll(rot(:X))(9)

Roller on 9 lines (9 blocks in total)
	1: Rx{Float64}: 0.0
	2: Rx{Float64}: 0.0
	3: Rx{Float64}: 0.0
	4: Rx{Float64}: 0.0
	5: Rx{Float64}: 0.0
	6: Rx{Float64}: 0.0
	7: Rx{Float64}: 0.0
	8: Rx{Float64}: 0.0
	9: Rx{Float64}: 0.0

In [11]:
dispatch!(l, params)

Roller on 9 lines (9 blocks in total)
	1: Rx{Float64}: 4.401944095106336
	2: Rx{Float64}: 1.5535204533843823
	3: Rx{Float64}: 1.4300312641677342
	4: Rx{Float64}: 4.523889290466054
	5: Rx{Float64}: 4.628532842653513
	6: Rx{Float64}: 6.2752354559564605
	7: Rx{Float64}: 4.805990563410075
	8: Rx{Float64}: 3.1411444182502724
	9: Rx{Float64}: 1.1663987397430668